# YOLOv11n Object Detection Demo
This notebook shows how to run the lightweight YOLOv11n detector on still images and a local webcam.

## 1. Install dependencies
Run the cell below if ultralytics or OpenCV are missing from your environment.

In [ ]:
!pip install -q ultralytics opencv-python matplotlib

## 2. Import libraries and load the model

In [ ]:
from pathlib import Path

import cv2
from IPython.display import display
from PIL import Image
from urllib.request import urlretrieve
from ultralytics import YOLO

MODEL_DIR = Path("models")
MODEL_DIR.mkdir(exist_ok=True)
MODEL_WEIGHTS = MODEL_DIR / "yolov8n.pt"


IMAGE_DIR = Path("demo_images")
IMAGE_DIR.mkdir(exist_ok=True)  # Keeps things organized for static assets

model = YOLO(str(MODEL_WEIGHTS))
device = getattr(model, "device", "cpu")
CLASS_ID_LOOKUP = {name: idx for idx, name in model.names.items()}
print(f"Loaded {MODEL_WEIGHTS.name} on {device}")
print(f"Drop sample images into {IMAGE_DIR.resolve()} before running detections.")


## 3. Download Ultralytics sample images (optional)
Use this helper to pull the stock demo photos (`bus.jpg`, `zidane.jpg`) directly from the public Ultralytics assets repository.

In [ ]:
ULTRALYTICS_SAMPLE_URLS = {
    "bus.jpg": "https://github.com/ultralytics/assets/releases/download/v0.0.0/bus.jpg",
    "zidane.jpg": "https://github.com/ultralytics/assets/releases/download/v0.0.0/zidane.jpg",
}


def download_ultralytics_samples(target_dir=IMAGE_DIR):
    target_dir.mkdir(exist_ok=True)
    for name, url in ULTRALYTICS_SAMPLE_URLS.items():
        destination = target_dir / name
        if destination.exists():
            print(f"{name} already exists, skipping.")
            continue

        print(f"Downloading {name} ...")
        urlretrieve(url, destination)

    print(f"Samples saved to {target_dir.resolve()}")


# Run once to make sure the stock demo images are available locally.
download_ultralytics_samples()


## 4. Detect objects in static images
Place any `.jpg`, `.png`, or `.jpeg` files in `demo_images/` and execute the cell to render detections inline.

In [ ]:
IMAGE_EXTENSIONS = {".jpg", ".jpeg", ".png", ".bmp"}

def run_on_images(conf=0.25):
    images = sorted([p for p in IMAGE_DIR.iterdir() if p.suffix.lower() in IMAGE_EXTENSIONS])
    if not images:
        print(f"Add some image files to {IMAGE_DIR.resolve()} and re-run this cell.")
        return

    for image_path in images:
        print(f"\nDetections for {image_path.name}")
        results = model.predict(image_path, conf=conf, verbose=False)
        annotated = results[0].plot()  # Returns a BGR NumPy array
        annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
        display(Image.fromarray(annotated_rgb))

run_on_images(conf=0.25)


## 5. Filter detections by class
This helper narrows detections to specific class names. The sample below keeps only the `tie` class for the stock `zidane.jpg` image.

In [ ]:
def run_filtered_detection(image_path, allowed_class_names, conf=0.25):
    missing = [name for name in allowed_class_names if name not in CLASS_ID_LOOKUP]
    if missing:
        raise ValueError(f"Unknown class names: {missing}")

    allowed_ids = [CLASS_ID_LOOKUP[name] for name in allowed_class_names]
    results = model.predict(image_path, classes=allowed_ids, conf=conf, verbose=False)
    annotated = results[0].plot()
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    display(Image.fromarray(annotated_rgb))


target_image = IMAGE_DIR / "zidane.jpg"
if target_image.exists():
    print(f"Showing only 'tie' detections for {target_image.name}")
    run_filtered_detection(target_image, ["tie"])
else:
    print(f"{target_image} is missing. Download the Ultralytics samples cell above and re-run.")


## 6. Detect objects from a local webcam
This helper streams frames from the camera, annotates them, and displays them in an OpenCV window. Press **q** to exit the stream.

In [ ]:
def run_webcam(conf=0.25, webcam_index=0, window_name="YOLOv11n Webcam"):
    cap = cv2.VideoCapture(webcam_index)
    if not cap.isOpened():
        print(f"Unable to open webcam index {webcam_index}")
        return

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            results = model.predict(frame, conf=conf, verbose=False)
            annotated = results[0].plot()
            cv2.imshow(window_name, annotated)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

# Uncomment to start streaming once you are ready.
# run_webcam(conf=0.25, webcam_index=0)
